In [ ]:
## load libraries
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import cmath as cm
from scipy import signal as sps
from IPython.display import HTML

# Introduction to Signal Processing

## Overview
The first part of this notebook will quickly go over several basic concepts from mathematics and physics that are important for signal processing. Nothing in this should be above college mathematics and physics, so if you feel confident you rember it, feel free to skip. 
1. [Complex numbers](#Complex-Numbers) are fairly essential to understanding how sinusoids are represented in signal processing (and electronics in general). They are also a big help for visualising amplitude and phase of a signal.
2. [Temporal Vs. Spectral Space](#Temporal-Vs.-Spectral-Space) will cover some concepts important to understanding what an electronic signal is and how to decompose it into it's components. The focus will lay on the Fourier series and transforms.
3. [Acquiring Data](#Acquiring-Data) will cover some of the basic concepts of data acquisition, such as the Nyquist-Shanon Theorem


# Complex Numbers
Why are we talking about complex numbers? The explanations go beyond the scope of this introduction, but the short version is that in engineering, a sinusoidal signal with time-invariant amplitude (A), angular frequency ($\omega$) and initial phase ($\varphi$) can be represented by a complex number called a phasor (short for phase vector). 

A quick and not quite exact explanation of these terms
- amplitude (A): the distance from the signals mid value to its maxima.
- angular frequency ($\omega$): ' to frequency, but indicating how quickly the signal goes through it's cycle. It is straightforward to compute from the frequency: $\omega = 2 \cdot \pi \cdot f$. If you remember your high school math, $2\pi$ in radians is a complete revolution (ie. $360^\circ$).
- frequency (f): how often the signals oscillates around its mid value.
- phase ($\varphi$): the delay in relation to an arbitrary point in the cycle of the signal, usually the starting point of another signal.

The same signal can be represented as:
- $x(t) = A \cdot \sin(\omega t + \varphi)$
- $x(t) = A \cdot e^{\imath(\omega t + \varphi)}$

These are equivalent. If you want to check this, use Euler's formula. A phasor is written without reference to time change, but only with reference to another signal, hence it is usually written as $A\cdot e^{\imath\cdot\varphi}$ or $A\angle\varphi$.

## The Imaginary Number $\imath$
You will have noted that these notations use the imaginary number $\imath = \sqrt{-1}$. This is the whole reason for this chapter. We will go over the most basic properties of complex numbers and how to use them. If you remember this from college, feel free to skip.

Note that in engineering, $\jmath$ is often used to represent the imaginary number in order to avoid confusion with the electric current, denotated *i*. This document should only use $\imath$, but my engineering training might get the better of me.

## Representation
There are two ways to represent complex number:
- *rectangular* or *cartesian*: decomposes the number into it's real and imaginary part and is written as $z = re + \imath \cdot im$ where $re$ and $im$ are, respectively, the real and imaginary parts of the number.
- *polar*: the number is represented as $magnitude \cdot e^{\imath\cdot argument}$ where the magnitude (r) (or modulus) is the distance from the origin (O) to the point and the argument describes the angle between the line going from the origin to z and the positive real axis.
Maybe you noticed that the polar representation uses a very similar representation as the phasor above. This is not a coincidence, $\varphi$ represents a delay in the cycle, so it is the same as the phase of our phasor. The magnitude represents the maximal value that can be projected on the real (or imaginary) axis and is the same as the amplitude of the phasor.

The way to represent a complex number I find the most helpful is to draw the number on the complex plane:

In [ ]:
# Prepare a complexe plane plot (aka argand diagram)
fig, ax = plt.subplots(1,2,figsize=(12,4),sharey='row',gridspec_kw={'width_ratios': [1, 2]})
ax[0].axis([-1,1,-1,1])
ax[0].spines['left'].set_position('center')
ax[0].spines['bottom'].set_position('center')
ax[0].spines['right'].set_color('none')
ax[0].spines['top'].set_color('none')
ax[0].add_artist(plt.Circle([0,0],1,color='k', fill=False))

## define a number and represent it as a vector on the complex plane
# pick an arbitrary phase
phi1 = 0
z1 = complex(np.cos(phi1),np.sin(phi1))
l1, = ax[0].plot([0,z1.real],[0,z1.imag],marker='o');
phi2 = np.pi/4 # 45° expressed in radians
z2 = complex(np.cos(phi2),np.sin(phi2))
l2, = ax[0].plot([0,z2.real],[0,z2.imag],marker='o');

## Let's plot the signals as a time series next to it
# Pick an arbitrary frequency
f = 50;
t = np.linspace(0,1/f,20) # 1/f, aka the periode of the signal (T), is one complete cycle.
x1 = np.sin(2*np.pi*f*t+phi1)
x2 = np.sin(2*np.pi*f*t+phi2)
ax[1].axis([0,1/f,-1,1])
ax[1].spines['bottom'].set_position('center')
ax[1].spines['right'].set_color('none')
ax[1].spines['top'].set_color('none')
plt.xlabel('t [s]', horizontalalignment='right', x=1)

l3, = ax[1].plot(t,x1)
l4, = ax[1].plot(t,x2);

In [ ]:
## Let's animate this
# using the same amount of samples for phi, t and the animation frequency will make things much simpler
Nframes = 20;
phi = np.linspace(0,2*np.pi,Nframes)
t = np.linspace(0,1/f,Nframes)
# recompute x1 and x2 with the new sample rate
x1 = np.sin(2*np.pi*f*t+phi1)
x2 = np.sin(2*np.pi*f*t+phi2)

def animate(i):
    z1 = complex(np.cos(phi[i]+phi1),np.sin(phi[i]+phi1))
    z2 = complex(np.cos(phi[i]+phi2),np.sin(phi[i]+phi2))
    l1.set_data([0,z1.real],[0,z1.imag])
    l2.set_data([0,z2.real],[0,z2.imag])
    l3.set_data(t[0:i],x1[0:i])
    l4.set_data(t[0:i],x2[0:i])

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=Nframes)


HTML(ani.to_jshtml())

As you can see, the phasor z1 lags behind z2 by a constant angle $\varphi_2$. The same difference can be observed in the time series: x1 laggs behind x2 by a constant delay. this delay can be computed $\Delta t = T\cdot{\varphi_2\over2\cdot\pi}$ where $T$ is the **period** of the signal and is the inverse of the frequency $T = {1\over f}$. It is the amount of time for a full cycle. In this example, we have:
\begin{equation*}
    \varphi_2 = 45° = {\pi\over4}\\ 
    T = {1\over f} = {1\over 50} = 0.02\ seconds
\end{equation*}
and therefore
\begin{equation*}
    \Delta t = {1\over50}\cdot{{\pi\over4}\over2\cdot\pi}={1\over50}\cdot{1\over8} = 0.0025\ seconds
\end{equation*}



## Operations with complex numbers
There are many mathematical relationships for complex numbers. There are two (well four) really important operations: addition and multiplication (substraction and division being, in a way, special cases of each respectively). This is one of the reason both representation for complex nubers (cartesian and polar) are used. It is very easy to add (or substract) in cartesian representation and it is very easy to multiply (or divide) in polar representation.

Given two complex numbers $x = x_r + \imath \cdot x_i$ and $y = y_r + \imath \cdot y_i$
\begin{equation}
x + y = (x_r + y_r) + \imath \cdot (x_i + y_i)\\
x \cdot y = x_r \cdot y_r + \imath \cdot x_r \cdot y_i + \imath \cdot x_i \cdot y_r + \imath^2 \cdot x_i \cdot y_i
\\
\ldots = (x_r \cdot y_r)-(x_i \cdot y_i) + \imath \cdot (x_r \cdot y_i + \cdot x_i \cdot y_r)
\end{equation}

Or, given two complex numbers $x = m_x \cdot e^{\imath\cdot \varphi_x}$ and $y = m_y \cdot e^{\imath\cdot \varphi_y}$

Using the notation
$$
    \Re a = \mathrm{the\ real\ part\ of\ } a =  m_a \cdot \cos(\varphi_a)\\
    \Im a = \mathrm{the\ imaginary\ part\ of\ } a =   m_a \cdot \sin(\varphi_a)\\
$$
We sum as with cartesian numbers
$$
x + y = \Re x + \Re y +\imath\cdot (\Im x + \Im y)
$$
and we convert this back to polar
$$
m_{xy} = \sqrt{(\Re x + \Re y)^2+(\Im x + \Im y)^2)}\\
\varphi_{xy} = \tan^{-1}\left({{\Im x + \Im y}\over{\Re x + \Re y}}\right)
$$
If you wisch to expand this, it's a good trigonometry exercise (make sure to check out the *angle sum and difference* trigonometric identities), but it's quite tedious.

# Temporal Vs. Spectral Space
In order to understand filtering, it is helpful to see signals as frequency components.
## Fourier Series
The Fourier series is a series of harmonically linked (all frequencies are a multiple of the lowest frequency) sinusoids that, once weighted and summed, are equivalent to an arbitrary signal for the period of the fundamental (ie. the lowest frequency present in the signal. Or, if the signal is of a determined length, one over the length of said signal).

To illustrate this, let us decompose a square wave signal at 440Hz with a duty cycle of 50%; a periodic signal oscillating  from +1 to -1 440 times per second where exactly half of the signal is positive  and half is negative.

In [ ]:
#This is the fundamental frequency. This is also the frequency of the first component.
fundamental = 440 
# Compute time and waveform data points
t = np.linspace(0,2/fundamental,1000);
x = sps.square(2*np.pi*440*t,0.5);
# show the waveform
fig, ax = plt.subplots(1,1,figsize=(12,4),sharex='col');

# Make the axes look nice
ax.axis([0,2.1/fundamental,-1.5,1.5]);
ax.set_xlabel('time [s]')
ax.set_ylabel('amplitude')
ax.set_xticks(np.arange(3)/fundamental)

# Draw the waveform
l1, = ax.plot(t,x,lw=4,color='k');

The Fourier series for square waveform $f(t)$ is computed as:
\begin{equation}
    f(x) = {4\over\pi}\sum_{n=1,3,5,\ldots}^{\infty}{1 \over n} sin({n \pi t \over T})
\end{equation}
In English, this means that the series is composed of all the odd components (since ${n=1,3,5,\ldots,\infty}$), where each is multiplied by $1/n$ (components get smaller and smaller) and the sum of the components is then multiplied by $4/\pi$ (ie. a bit over 1). This Formula derives from the Fourier transform, which allows to analytically decompose a signal into its Fourier series. This transform is not explained here; Imagine a blackbox operation that converts a linear equation into a sum of frequency components. In real world application algorithms such as the fast Fourier transform (FFT) are used, which are very good at approximating the discrete (ie. digital) Fourier transform.

In [ ]:
# Compute The Fourier series components
# Nframes determines how many frames the animation will have 
# each frame will add one more component, hence the number of
# frames is also the number of components to be computed
Nframes = 10;
# Set up empty line to accept the series' data
l2, = ax.plot([],[],lw=4,color='b');
# Set up empty lines to accept the components data
lines = [ax.plot([],[]) for _ in range(Nframes)];

# Add textual information on root mean square deviation
# and number of components
text = ax.text(0.0002,-1,"RMSD = {:.3}\n # of components = {:}".format(np.sqrt(np.mean(np.square(x))),0));

# Compute the components according to equation above
x_components = [4/np.pi*1/n*np.sin(2*np.pi*fundamental*n*t) for n in range(1,Nframes*2+1,2)];

# Define animation update function
def animate(i):
    # at frame 0, remove all components
    if i==0:
        for line in lines:
            line[0].set_data([],[]);
    #recompute the fourier series to the ith component
    x_series = sum(x_components[0:(i+1)],0);
    # plot series
    l2.set_data(t,x_series);
    # add new component to graph
    lines[i][0].set_data(t,x_components[i]);
    # update text
    x_rmsd = np.sqrt(np.mean(np.square(x-x_series)));
    text.set_text("RMSD = {:.3}\n # of components = {:>}".format(x_rmsd,i+1));
    
# Create animation object
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=Nframes, interval = 500);
# Run animation
HTML(ani.to_jshtml())

As the number of components increases, the series approximates the original better and better. The *root mean square deviation* (an indicator of the quality of the fit, lower is better) decreases in a logarithmic way. 

## Spectral representation
No, spectral has nothing to do with ghosts. The specter (or spectrum) of a signal is it's frequency signature. Typically, this is represented in the form of a power spectrum density (**PSD**): a frequency vs. power density plot. The amplitude of the specter at a given frequency tells how much of the power is contained at that frequency. Let's look at a few examples for better understanding:

In [ ]:
# Initialize variables
# frequency of the periodic signals
f_signal = 4
# sampling rate
sr = 68

# timeseries vector of length 4*T (two periods(T), T = 1/f)
t = np.arange(0,4*1/f_signal,1/sr)

Each signal will be plotted in temporal space on the left and twice in spectral space, once as it's theoretical expression and once using the PSD computed with `scipy`'s `signal.periodogram` function.

### Dirac delta function ($\delta$)
$\delta$ is a very important function for signal processing. It is the input for a so called *impulse response* which helps to characterize electronic systems. It is also frequently present in the spectra of common signals. We will use it several times further down.

The Dirac delta function is defined as:
\begin{equation*}
    \delta(t) = \left\{\begin{matrix}
     +\infty  & if\ t = 0\\ 
     0 & if\ t\neq0 
    \end{matrix}\right.
\end{equation*}
and fulfils the condition
$$
\int_{-\infty}^{\infty}{\delta(t)dt} = 1
$$
In other words, it is a spike with an infinite amplitude, a width of 0 and a surface area of 1. This is of course impossible to use in the digital world, therefore it is expressed as:
\begin{equation*}
    \delta(n) = \left\{\begin{matrix}
     sr  & if\ n = 0\\ 
     0 & if\ n\neq0 
    \end{matrix}\right.
\end{equation*}
where $n$ is the sample number and sr is the sample rate. This creates a square pulse of amplitude sr and of width $1\over sr$. This gives us an area under the curve of $sr \cdot {1\over sr} = 1$.


In [ ]:
# Initialize figure
from scipy.signal import periodogram
fig, ax = plt.subplots(1,3,figsize=(12,4));
# Direct current(DC)
x = [0 if x !=0 else sr for x in t]
f = [0,32]
X = [1,1]
fp,Xp = periodogram(x,fs=sr,detrend=False)

ax[0].axis([0,1,0,1.1])
ax[0].plot(t,x)
ax[1].axis([-1,sr/2,10**-2,3])
ax[1].plot(f,X)
ax[1].set_yscale('log')
ax[2].axis([-1,sr/2,10**-2,3])
ax[2].plot(fp,Xp)
ax[2].set_yscale('log')

First, let's look at the axes: 
- The amplitude of a periodogram is usually expressed on a logarithmic scale because we are usually interested in order of magnitude differences.
- The frequencies axis ranges from 0 to $sr/2$. This is a convenient amount of data to show since it covers everything below Nyquist's frequency (we'll get to that shortly). It is not the only reason, it has also to do with the use of `scipy.signal.periodogram` and more.

Now let's look at the data. The theoretical PSD would be a constant amplitude of 1 across all frequencies. We see that the periodogram is a bit off, ranging from amplitude 1 to 2. This is because it only got a very short window of data to work with, $\delta$ is supposed to range from $-\infty$ to $+\infty$. But again, we are mostly interested in order of magnitude differences, so a factor of 2 is not that much off.
### Direct current(DC)
Direct current is a signal that remains at a constant (and therefore does not vary over time). It's PSD is entirely concentrated at f=0

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(12,4));

# Direct current(DC)
x = [1 for x in t]
f = [0]
X = [1]
fp,Xp = periodogram(x,fs=sr,detrend=False)

ax[0].axis([0,1,0,1.1])
ax[0].plot(t,x)
ax[1].axis([-1,sr/2,10**-2,2])
ax[1].stem(f,X,use_line_collection=True)
ax[1].set_yscale('log')
ax[2].axis([-1,sr/2,10**-2,2])
ax[2].stem(fp,Xp,use_line_collection=True)
ax[2].set_yscale('log')

The transform of a DC signal is $1 \cdot \delta(0)$, ie. all the power (the  1) is concentrated at f=0Hz. So a DC signal is nothing but a "periodic" signal that does not change in time. It is often called the DC component of a signal. removing the DC component is the same as to center the signal on the non temporal axis such as $\int_{-\infty}^{\infty}{f(t)dt} = 0$. In the code above we run `fp,Xp = periodogram(x,fs=sr,detrend=False)`; the `detrend=False` part preserves the DC component in the periodogram.

### Sinusoid
Let's look at the sin function: $f(t) = A \cdot \sin(\omega\cdot (t + \varphi))+c$ where:
- A is the amplitude
- $\omega$ is the angular speed in radians per second. It is $\omega = 2 \cdot \pi \cdot f$.
- $\varphi$ is the phase. This is equivalent to a shift on the temporal axis.
- c is a constant value, ie. the DC component. It is equivalent to a shift on the amplitude axis.

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(12,4));
# Sinusoid
x = np.sin(2*np.pi*f_signal*t)
f = [f_signal]
X = [0.5]
fp,Xp = periodogram(x,fs=sr,detrend=False)

ax[0].axis([0,1,-1.1,1.1])
ax[0].plot(t,x)
ax[1].axis([-1,sr/2,10**-2,2])
ax[1].stem(f,X,use_line_collection=True)
ax[1].set_yscale('log')
ax[2].axis([-1,sr/2,10**-2,2])
ax[2].stem(fp,Xp,use_line_collection=True)
ax[2].set_yscale('log')

The transform of a sinusoid is a dirac at the frequency of the sinusoid. This is rather obvious if you consider that what we are representing in the PSD is a series of sin functions that if summed up give the temporal signal. Decomposing a sin will obviously return the sin itself. 

There is something  weird though, it only has amplitude of 0.5. This is because we only see a half truth here (quite literally). A spectrum always runs from $-\infty$ to $+\infty$. In truth, there are two $\delta$ with amplitude 0.5 at $-f_{signal}$ and $f_{signal}$. It is rather common to only represent the positive party of a PSD since the negative part is identical.

### sinc (sine cardinal)
The sinc function is often encountered when switching from temporal to spectral space (and vice versa). it is defined as:
$$
sinc(x) = \left\{\begin{matrix}
 1  & if\ x = 0\\ 
 {\sin(x)\over{x}} & if\ x\neq0 
\end{matrix}\right.
$$
It is also called the sampling function or, very colloquially, the Mexican  hat function because of how it looks. We have to define its value at $x = 0$ to avoid a zero division. This is the definition for the normalized sinc function, we will be using a non normalized version where $x = \omega t$.

For this one we'll use $-1 \le t \le 1$ since the "hat" is centered at $t=0$.

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(12,4));
t1 = np.arange(-1,1,1/sr)
x = [np.sin(2*np.pi*f_signal*x)/(2*np.pi*f_signal*x) if x !=0 else 1 for x in t1]
f = [-f_signal/2,f_signal/2,f_signal/2*1.00000000000001,32]
X = [1/f_signal,1/f_signal,0,0]
fp,Xp = periodogram(x,fs=sr,detrend=False,return_onesided=False)

ax[0].axis([-1,1,-1.1,1.1])
ax[0].plot(t1,x)
ax[1].axis([0,sr/2,10**-4,1])
ax[1].plot(f,X)
ax[1].set_yscale('log')
ax[2].axis([0,sr/2,10**-4,1])
ax[2].stem(fp,Xp,use_line_collection=True)
ax[2].set_yscale('log')

The  transform is a square with amplitude $1\over{f_{signal}}$ and width of$f_{signal}$ (remember that half of it is in the negative frequencies). Note that the periodogram is quite off the mark. This is because the width and amplitude actually depend on the sampling frequency.
Interestingly enough, the transform of a square function (not a periodic square wave, a single square pulse) is a sinc.

### square wave 
Finally, let's look at the square wave from the Fourier decomposition above.

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(12,4));
t = np.arange(0,1,1/sr)            

x = sps.square(2*np.pi*f_signal*t,0.5)/2;
f = np.arange(1,int(sr/2))*f_signal
X = np.array([(2/n/np.pi)**2*np.abs(np.sin(n*np.pi/2)) for n in range(1,int(sr/2))]);
fp,Xp = periodogram(x,sr)

ax[0].axis([0,1,-0.55,0.55])
ax[0].plot(t,x)
ax[1].axis([0,32,10**-4,1])
ax[1].stem(f,X,use_line_collection=True);
ax[1].set_yscale('log')
ax[2].axis([0,32,10**-4,1])
ax[2].stem(fp,Xp,use_line_collection=True)
ax[2].set_yscale('log')

The PSD should be a comb of $\delta$ with amplitude $2\over{n \pi}$ for all odd n. Again, the periodogram has a hard time to compute this; the signal is too short and the sampling rate too low.

# Acquiring Data

## What is a signal
In neuroscience, most signals are **voltage varying in time**. Many bio-signals are amplified and filtered electric potentials (ie. voltages), such as ECG, EEG, EMG, etc. Others might be the result of a transducer, a device converting different physical measurements (eg. distance, pressure) to (usually) voltage.
A notable exception are all video based signals, such as eye tracking. These are usually recorded as some arbitrary units directly in digital form.
### Analog to Digital
Most signals are measured as analog signals. This means they are continuous over time. Computers can't store or even process analog signals, we therefore use an *analog to digital converter* (ADC) which does what it says on the tin. This step is called **discretization**. From here on, continuous and analog as well as discrete and digital are going to be used interchangeably.
Here is an example of a pure sine wave discretized at different sampling rates.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate


# We'll use this interpolation to rebuild an analogue signal from digital samples
# credit for this implementation goes to endolith: https://gist.github.com/endolith
def sinc_interp(x, s, u):
    """
    Interpolates x, sampled at "s" instants
    Output y is sampled at "u" instants ("u" for "upsampled")
    
    from Matlab:
    http://phaseportrait.blogspot.com/2008/06/sinc-interpolation-in-matlab.html        
    """
    # Validate arguments
    if len(x) != len(s):
        raise(Exception, 'x and s must be the same length')
    
    # Find the period    
    T = s[1] - s[0]    
    
    transposed = np.transpose(np.tile(np.arange(len(s)), (len(u),1))) * T    
    
    sincM = np.tile(u, (len(s), 1)) - transposed
    y = np.dot(x, np.sinc(sincM / T))
    
    return y
        

#Set up figure
fig, axes = plt.subplots(3,4,figsize=(12,4),sharex='col',sharey='row')

# The signal will be a pure sinusoidal, at a frequency of 3 Hz (Hertz, Hz = 1/s)
frequency = 3;
# A high sampling rate (number of recorded data points per time unit) will give the
# illusion of an analog signal, you will not be able to see the time steps.
# At a lower sampling rate, you can clearly see when a point was sampled
# If the sampling rate is at or below the Nyquist frequency, the acquired signal will be erroneous.

# Now lets discretize this with a sampling rate of 10 Hz

# We'll look at a dozen frequencies
sampling_rates = [50,25,12,7,6.2,6,5.8,5,4.5,4,3,2.9];#np.logspace(np.log10(96),np.log10(1.5),num=8);

# Compare all signals to a sampling rate of 50 Hz. Used by the interpolation
reference_sampling_rate = 50;
reference_time = np.arange(-10,10,1/reference_sampling_rate);
reference_time = np.append(reference_time,1);
reference_voltage = np.sin(2*np.pi*frequency*reference_time)

# Iterate over axes and sampling rates
for (axe,sampling_rate) in zip(axes.reshape(-1),sampling_rates):
    # Create the time vector.
    time = np.arange(-10.01,10,1/sampling_rate);
    
    # Compute the sinusoidal (x = Amplitude*sin(angular_frequency * time))
    # Where angular_frequency = 2 * pi * frequency
    voltage  =  np.sin(2*np.pi*frequency*time);
    
    # In order to get a better idea of the signal represented by the samples it is interpolated
    # and plotted on top of the discretized signal. this step is not part of discretization.
    # interpolate the signal using Whittaker-Shannon Interpolation
    interpolated_voltage = sinc_interp(voltage,time,reference_time);
    
    axe.stem(time,voltage,use_line_collection=True);
    axe.plot(reference_time,interpolated_voltage,'r');
    axe.plot(reference_time,reference_voltage,'g.')
    axe.set_title('Sampling Rate = {:05.2f} Hz'.format(sampling_rate))
    axe.set_xlim(0,1)
    

plt.tight_layout()

These plots show us that there is a qualitative change in the interpolated signal when the sampling rate drops below 6 Hz. While at 6 Hz the signal still looks more or less like the original (well, kinda), at 5.8 the interpolated signal suddenly is in antiphase. Even lower, completely new signals with much lower frequency appear. Notably, at 3 Hz (the fundamental frequency of the original signal) the signal completely flattens out to a DC (Direct Current) Signal. This makes sense, since each time we take a sample, the signal will be at exactly the same amplitude.

## The Nyquist-Shanon Theorem
These effects are called aliasing and are caused by a sampling rate that does not comply with the *Whittaker-Nyquist-Kotelnikov-Shannon* theroem. There were multiple, independent, discoveries of this theorem. It is most commonly known as the *Nyquist* or the *Nyquist-Shannon* Theorem. 

In the words of Claude Shannon:

>If a function x(t) contains no frequencies higher than B Hertz, it is completely determined by giving its ordinates at a series of points spaced 1/(2B) seconds apart

In other words, a signal is completely determined (ie. there is no information lost) if the sampling rate is more than twice as high as the highest frequency component present in the signal. This frequency of *2B* is called the Nyquist rate (or frequency). In the example it is 6Hz since the highest frequency (and only) component is at 3 Hz.

A look at the plots shows that there are some issues: frequencies complying  with the theorem but close to the Nyquist rate are still quite distorted. We use the best possible interpolation method: the sinc interpolation (named after the sinc function described above). But we have only sampled over a limited time window. Would we extend it to $\pm \infty$ we should be able to reconstruct the signal perfectly. In theory.

The point of these sampling theorems is not just to avoid losing information. 
At 6 and 3 Hz the signal flatlines, this is due to the sampling rate being at or at twice, the frequency of the signal. In this case, we sampled every time the signal passed through 0. So ignoring the theorem can lead to completely asinine  results.

Here is an auditory example. 

**WARNING: check your sound volume: loud high pitched sounds**

We'll use a perfect fifth harmony in two octaves:C7 (2093Hz) - F8 (2794Hz) and C8 (4186Hz) - F8 (5588Hz).

First, we'll sample it at 44100 Hz. This is the typical sample rate used for music.


In [ ]:
import numpy as np
import simpleaudio as sa

# define a playback duration
note_length = 1
# define sampling rate
sr = 44100
# compute the frequencies of notes
f = [pow(2,(96-69)/12)*440,pow(2,(101-69)/12)*440,
    pow(2,(108-69)/12)*440,pow(2,(113-69)/12)*440]
# define some variable amplitudes for the notes
amplitudes = np.array([1,1,0.8,0.8])/len(f);
# compute time vector
t = np.arange(0,note_length,1/sr)
# compute audio signal
signal = np.sum(np.array([a*np.sin(2*np.pi*ff*t) for a in amplitudes for ff in f]),0)/len(f)
audio = signal * (2**15 -1)
audio = np.int16(audio)
play_obj = sa.play_buffer(audio,1,2,sr)
play_obj.wait_done()

Nothing special so far. now let's crank the sampling rate way down to 11025 Hz. This is typically what you would hear on a phone line

In [ ]:
# define sampling rate
sr = 11025
# compute time vector
t = np.arange(0,note_length,1/sr)
# compute audio signal
signal = np.sum(np.array([np.sin(2*np.pi*ff*t) for ff in f]),0)/len(f)
audio = signal * (2**15 -1)
audio = np.int16(audio)
play_obj = sa.play_buffer(audio,1,2,sr)
play_obj.wait_done()

Even though nothing major happened, there is an issue with the higher notes. There is a distinctive electronic flair to it. This is the aliasing; due to the improper sampling rate, completely new frequencies are introduced to the signal. 

Now let's go all the way down to 8000 Hz. You might encounter this on walkie-talkies.

In [ ]:
# define sampling rate
sr = 8000
# compute time vector
t = np.arange(0,note_length,1/sr)
# compute audio signal
signal = np.sum(np.array([np.sin(2*np.pi*ff*t) for ff in f]),0)/len(f)
audio = signal * (2**15 -1)
audio = np.int16(audio)
play_obj = sa.play_buffer(audio,1,2,sr)
play_obj.wait_done()

Now the pitch is hardly identifiable. It sounds more like a modem from the 90's than a perfect fifth harmonic.